In [1]:
#This is following the Summarization Tutorial for Transformers found on HuggingFace website
#https://huggingface.co/docs/transformers/tasks/summarization

In [2]:
pip install transformers datasets evaluate rouge_score


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 3.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Obtaining dependency information for absl-py from https://files.pythonhosted.org/packages/01/e4/dc0a1dcc4e74e08d7abedab278c795eef54a224363bb18f5692f416d834f/absl_py-2.0.0-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 25.0 MB/s eta 0:00:00a 0:00:01
  Obtaining dependency information for click from https://files.pythonhosted.org/packages/00/2e/d53fa4befbf2cfa713304affc7ca780ce4fc1fd8710527771b58311a3229/click-8.1.7-py3-none-any.whl.metadata
  Obtaining dependency information for joblib from https://files.pythonhosted.org/packages/10/40/d551139c85db202f1f384ba8bcf96aca2f329440a844f924c8a0040b6d02/joblib-1.3.2-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━

In [3]:
from datasets import load_dataset

# Load the BillSum dataset from hugging face
billsum = load_dataset("billsum", split="ca_test")

Generating ca_test split: 100%|██████████| 1237/1237 [00:00<00:00, 22608.09 examples/s]


In [4]:
# Split into Train and Test Set 

#Test size is proportion to include in test split
# Most is going to be used to train our model
billsum = billsum.train_test_split(test_size=0.2)

In [5]:
billsum

DatasetDict({
    train: Dataset({
        features: ['text', 'summary', 'title'],
        num_rows: 989
    })
    test: Dataset({
        features: ['text', 'summary', 'title'],
        num_rows: 248
    })
})

In [6]:
from transformers import AutoTokenizer 

tokenizer = AutoTokenizer.from_pretrained("t5-small")

In [7]:
# We want T5 to know we are summarizing

prefix = "summarize: "

def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["text"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)

    labels = tokenizer(text_target=examples["summary"], max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [9]:
tokenized_billsum = billsum.map(preprocess_function, batched=True)

Map: 100%|██████████| 248/248 [00:00<00:00, 823.91 examples/s]


In [11]:
# Create a batch of examples

from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model="t5-small")

In [12]:
# Include a metric to evaluate your models performance

import evaluate

rouge = evaluate.load("rouge")

In [13]:
#Compute the Rogue Metric for the Model

import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}


# Time to Train the Model!

In [14]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")

In [20]:
training_args = Seq2SeqTrainingArguments(
    output_dir="sample_billsum_model",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=4,
    predict_with_generate=True,
    fp16=False,
    push_to_hub=False,
)

In [24]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_billsum["train"],
    eval_dataset=tokenized_billsum["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

# Please USE CUDA to Train this Model. I am on Mac so I don't have access to a GPU. I might rent a cloud gpu!

In [25]:
trainer.train()

 25%|██▌       | 62/248 [05:00<13:58,  4.51s/it]/Volumes/areeb-ssd/SideProjects/Sentence-Modifier-w-GenerativeTransformer/.env/lib/python3.11/site-packages/transformers/generation/utils.py:2507: UserWarning: MPS: no support for int64 min/max ops, casting it to int32 (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/mps/operations/ReduceOps.mm:1271.)
  if unfinished_sequences.max() == 0:
                                                
 25%|██▌       | 62/248 [05:33<13:58,  4.51s/it]

{'eval_loss': 2.725334405899048, 'eval_rouge1': 0.1284, 'eval_rouge2': 0.0376, 'eval_rougeL': 0.1051, 'eval_rougeLsum': 0.105, 'eval_gen_len': 19.0, 'eval_runtime': 33.586, 'eval_samples_per_second': 7.384, 'eval_steps_per_second': 0.476, 'epoch': 1.0}


                                                 
 50%|█████     | 124/248 [12:53<19:37,  9.50s/it]

{'eval_loss': 2.5248260498046875, 'eval_rouge1': 0.137, 'eval_rouge2': 0.0465, 'eval_rougeL': 0.1128, 'eval_rougeLsum': 0.1128, 'eval_gen_len': 19.0, 'eval_runtime': 32.1068, 'eval_samples_per_second': 7.724, 'eval_steps_per_second': 0.498, 'epoch': 2.0}


                                                 
 75%|███████▌  | 186/248 [19:09<08:40,  8.39s/it]

{'eval_loss': 2.46444034576416, 'eval_rouge1': 0.1406, 'eval_rouge2': 0.0498, 'eval_rougeL': 0.1149, 'eval_rougeLsum': 0.1149, 'eval_gen_len': 19.0, 'eval_runtime': 29.6516, 'eval_samples_per_second': 8.364, 'eval_steps_per_second': 0.54, 'epoch': 3.0}


                                                 
100%|██████████| 248/248 [27:18<00:00,  6.61s/it]

{'eval_loss': 2.447948932647705, 'eval_rouge1': 0.1431, 'eval_rouge2': 0.05, 'eval_rougeL': 0.116, 'eval_rougeLsum': 0.116, 'eval_gen_len': 19.0, 'eval_runtime': 28.9727, 'eval_samples_per_second': 8.56, 'eval_steps_per_second': 0.552, 'epoch': 4.0}
{'train_runtime': 1638.4635, 'train_samples_per_second': 2.414, 'train_steps_per_second': 0.151, 'train_loss': 2.9873620310137348, 'epoch': 4.0}


TrainOutput(global_step=248, training_loss=2.9873620310137348, metrics={'train_runtime': 1638.4635, 'train_samples_per_second': 2.414, 'train_steps_per_second': 0.151, 'train_loss': 2.9873620310137348, 'epoch': 4.0})

In [27]:
trainer.save_model("./sample_billsum_model/")

In [22]:
#The Model is now trained

## Lets Test our New Model!

In [35]:
from transformers import pipeline
text = "summarize: The Inflation Reduction Act lowers prescription drug costs, health care costs, and energy costs. It's the most aggressive action on tackling the climate crisis in American history, which will lift up American workers and create good-paying, union jobs across the country. It'll lower the deficit and ask the ultra-wealthy and corporations to pay their fair share. And no one making under $400,000 per year will pay a penny more in taxes."

summarizer = pipeline("summarization", model='./sample_billsum_model/', max_length=100)
summarizer(text)

[{'summary_text': "the Inflation Reduction Act lowers prescription drug costs, health care costs, and energy costs . it's the most aggressive action on tackling the climate crisis in American history . no one making under $400,000 per year will pay a penny more in taxes ."}]